Условие:

В отделениях
установлены терминалы электронной очереди. <br>
Для обслуживания, клиент берет талон. Один талон – один клиент. <br>
Талон имеет категорию и в зависимости от нее может быть обслужен либо у менеджера по обслуживанию (МО), либо у менеджера по <br> продажам (МП). <br>

Какое обслуживание требуется определяет поле CURR_OPCAT_ID (категория талона). <br>
Соответствие роли и категории талона указано в файле OPCAT_ID_HIST_mapping.xlsx <br>
При этом, в разные года значение этого поля может меняться. <br>
Файл opcat_hist.csv хранит историю изменений для значений поля CURR_OPCAT_ID. <br>

Необходимо построить модель, прогнозирующую клиентопоток (сколько клиентов придет на обслуживание в каждый час) для каждого <br> отделения банка к каждому менеджеру. <br>

In [1]:
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.model_selection import train_test_split,TimeSeriesSplit
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder,LabelBinarizer,LabelEncoder,PolynomialFeatures

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor, Pool

from sklearn.linear_model import LinearRegression, Ridge, LassoCV
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

import lightgbm as lgb

from sklearn.svm import SVR

### Считываем данные

In [18]:
tickets132 = pd.read_csv('tickets/tickets_2013_02.csv', sep=';',encoding ='utf8')
tickets141 = pd.read_csv('tickets/tickets_2014_01.csv', sep=';',encoding ='utf8')
tickets142 = pd.read_csv('tickets/tickets_2014_02.csv', sep=';',encoding ='utf8')
tickets151 = pd.read_csv('tickets/tickets_2015_01.csv', sep=';',encoding ='utf8')
tickets152 = pd.read_csv('tickets/tickets_2015_02.csv', sep=';',encoding ='utf8')
opcat_hist=pd.read_csv('opcat_hist.csv', sep=';',encoding ='utf8')
opcat_map=pd.read_csv('OPCAT_ID_HIST_mapping.csv', sep=';',encoding ='utf8')

In [3]:
tickets=tickets132.append([tickets141,tickets142,tickets151,tickets152])

### Преобразуем данные

<br>
CURR_COUNTER - номер окошка в котором принимают клиента<br>
USER_ID - id менеджера<br>
TICKET_WAIT_TIME,TICKET_SERV_TIME - время ожидания и обслуживания<br>

Эти параметры нам не нужны для определения кол-ва клиентов к каждому типу менеджеров<br>

EVENT_TYPE_ID - id действия по билету <br>
Для нашей задачи достаточно взять строчки с 'EVENT_TYPE_ID'==1 (получение билета), так как нам нужно именно понять сколько клиентов пришло в отделение в определенный час (а не их последующие действия)

In [4]:
tickets=tickets.drop(['CURR_COUNTER','USER_ID','TICKET_WAIT_TIME','TICKET_SERV_TIME'],axis=1)
tickets=tickets[tickets['EVENT_TYPE_ID']==1].drop(['EVENT_TYPE_ID'],axis=1)

In [5]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1593063 entries, 0 to 985002
Data columns (total 4 columns):
SUBBRANCH_ID     1593063 non-null int64
CURR_OPCAT_ID    1593063 non-null int64
EVENT_DTTM       1593063 non-null object
TICKET_ID        1593063 non-null object
dtypes: int64(2), object(2)
memory usage: 60.8+ MB


В таблице истории категорий переводим время в формат datetime (5999 переходит в NaT)<br>
Выбираем те строчки, где EFFECTIVE_TO_DTTM обратилось в NaT, это как раз строчки в которых указана граничная дата, после которой изменялась категория (для этих данных)<br>
Далее обнуляем даты, где ACTIVE_FLG==1, это значит что данная категория не поменялась и нам не нужна (еще есть 2 категории которые просто отключили у них  ACTIVE_FLG==0 , но нет маппинга на новую категорию)<br>

Все это я делаю для того чтобы определить когда категории, которые перестали быть активными, были заменены на новые. Это нужно чтобы понять в разные промежутки времени к какому менеджеру отнести ту или иную категорию
Сначала я смотрю где EFFECTIVE_TO_DTTM = 5999, это строчки с последними изменениями данной категории
Затем обнуляю EFFECTIVE_FROM_DTTM  в строчках  где ACTIVE_FLG=1 , я их в дальнейшем не использую для маппинга, потому что эти категории действуют до сих пор и не менялись на другие категории. Использую в итоге строчки в которых ACTIVE_FLG=0 , это как раз те категории, которые превратились в новые и для них важно для какой даты мы будем вычислять менеджера﻿.

In [19]:
opcat_hist


,OPCAT_ID,OPCAT_NAME,EFFECTIVE_FROM_DTTM,EFFECTIVE_TO_DTTM,ACTIVE_FLG,DELETED_FLG
0,-99,По всем операциям,01.01.1900,01.06.2012,1,1
1,-99,По всем операциям,01.06.2012,23.08.2013,1,0
2,-99,По всем операциям,23.08.2013,31.12.5999,1,0
3,1,Вклады / Металлические счета / НПФ,01.01.1900,01.06.2012,1,1
4,1,Вклады / Металлические счета / НПФ,01.06.2012,23.08.2013,1,0
5,1,Вклады / Металлические счета / НПФ,17.10.2013,31.12.5999,0,0
6,1,Вклады / Металлические счета / НПФ,23.08.2013,17.10.2013,1,0
7,2,Банковские / Кредитные карты,01.01.1900,01.06.2012,1,1
8,2,Банковские / Кредитные карты,01.06.2012,04.12.2012,1,0
9,2,Банковские / Кредитные карты,04.12.2012,23.08.2013,1,0


In [6]:
opcat_hist['EFFECTIVE_FROM_DTTM']=pd.to_datetime(opcat_hist['EFFECTIVE_FROM_DTTM'],format='%d.%m.%Y')
opcat_hist['EFFECTIVE_TO_DTTM']=pd.to_datetime(opcat_hist['EFFECTIVE_TO_DTTM'],  errors='coerce',format='%d.%m.%Y')
opcat_hist=opcat_hist[opcat_hist['EFFECTIVE_TO_DTTM'].isnull()].sort_values(by=['OPCAT_ID','EFFECTIVE_FROM_DTTM'])
opcat_hist.loc[opcat_hist['ACTIVE_FLG']==1,'EFFECTIVE_FROM_DTTM']=pd.NaT

In [16]:
opcat_hist

,OPCAT_ID,OPCAT_NAME,EFFECTIVE_FROM_DTTM,EFFECTIVE_TO_DTTM,ACTIVE_FLG,DELETED_FLG
2,-99,По всем операциям,NaT,NaT,1,0
5,1,Вклады / Металлические счета / НПФ,2013-10-17,NaT,0,0
10,2,Банковские / Кредитные карты,2013-10-17,NaT,0,0
14,3,Оформление Кредитов,2014-09-03,NaT,0,0
21,4,Оформление Ипотеки,2014-09-03,NaT,0,0
28,5,Индивидуальные сейфы,2014-04-15,NaT,0,0
34,6,Платежи,2013-10-17,NaT,0,0
38,7,"Прием и выдача наличных, пенсии",2013-10-17,NaT,0,0
42,8,Денежные переводы,2013-10-17,NaT,0,0
47,9,Валютно-обменные операции,2013-10-17,NaT,0,0


Достаем столбец дат

In [7]:
opcat_dtm=opcat_hist.sort_values(by='OPCAT_ID')['EFFECTIVE_FROM_DTTM']

В маппинге категорий сортируем и добавляем столбец дат <br>
Удаляем 1 строчку с 99 категорией<br>
Заменяем роли на цифровые значения

In [8]:
new_opcat_map=opcat_map[['OPCAT_ID_HIST','opcat_id_new','POS_PRIZNAK']].sort_values(by='OPCAT_ID_HIST')
new_opcat_map['DATE']=opcat_dtm.values
new_opcat_map.drop(new_opcat_map.index[0],inplace=True)
new_opcat_map['DATE']=new_opcat_map['DATE'].dt.date
new_opcat_map['POS_PRIZNAK']=new_opcat_map['POS_PRIZNAK'].map({'МП':1,'не МП':0})
new_opcat_map['POS_PRIZNAK']=new_opcat_map['POS_PRIZNAK'].astype(int)

In [20]:
new_opcat_map

,OPCAT_ID_HIST,opcat_id_new,POS_PRIZNAK,DATE
0,1,16,1,2013-10-17
2,2,12,1,2013-10-17
10,3,17,1,2014-09-03
4,4,20,1,2014-09-03
12,5,24,0,2014-04-15
15,6,26,0,2013-10-17
7,7,15,0,2013-10-17
28,8,14,0,2013-10-17
24,9,31,0,2013-10-17
21,10,25,0,2013-10-17


Создаем словарь для будущего маппинга с категорией, граничной датой и ролью

In [9]:
role_map={}
for row in new_opcat_map.itertuples(index=False):
    role_map[row.OPCAT_ID_HIST]=[row.opcat_id_new,row.POS_PRIZNAK,row.DATE]

Функция маппинга для простановки правильных ролей в tickets

In [10]:
def role_mapping(data,map_dict):
    result_arr=np.array([])
    for row in data:        
        category=row[0]
        bound = map_dict[category][2]
        new_category=map_dict[category][0]
        if pd.isnull(bound):
            result_arr=np.append(result_arr,map_dict[category][1])
        elif row[1]<bound:
            result_arr=np.append(result_arr,map_dict[category][1])
        else:
            result_arr=np.append(result_arr,map_dict[new_category][1])
    return result_arr

Создаем вспомогательные столбцы <br>
Считаем кол-во посетителей в каждый час для каждой категории

In [11]:
tickets['EVENT_DTTM']=pd.to_datetime(tickets['EVENT_DTTM'], format="%d.%m.%Y %H:%M:%S")
tickets['DATE']=tickets['EVENT_DTTM'].dt.date
tickets['HOUR']=tickets['EVENT_DTTM'].dt.hour
count_tickets=tickets.groupby(['SUBBRANCH_ID','DATE','HOUR','CURR_OPCAT_ID'])['TICKET_ID'].count()
count_tickets=pd.DataFrame({'COUNT':count_tickets}).reset_index()

Достаем даты и категории для получения актуальных ролей <br>
Применяем ф-ию маппинга и формируем столбец ROLE<br>
Считаем кол-во людей каждый час по каждой роли<br>

In [12]:
np_data=count_tickets[['CURR_OPCAT_ID','DATE']].values
count_tickets['ROLE']=role_mapping(np_data,role_map)
amount=count_tickets.groupby(['SUBBRANCH_ID','DATE','HOUR','ROLE'])['COUNT'].sum()
amount=pd.DataFrame({'AMOUNT':amount}).reset_index()
amount['ROLE']=amount['ROLE'].astype(int)

Для целостности данных нужно чтобы каждый час была информация по Обоим ролям, для этого<br>
нужно добавить вторую роль там, где получилась только 1 роль для часа<br>
Выбираем строчки у которых только 1 роль, удваиваем их (делаем по 1 строчке для каждой роли) и присваиваем 0 кол-во людей<br>
Добавляем эту таблицу снизу к основной, удаляем дубли и сортируем<br>
В итоге к каждой единичной роли добавилась противоположная с 0 кол-вом людей

In [13]:
absent_roles=amount.groupby(['SUBBRANCH_ID','DATE','HOUR'])['ROLE'].count()
absent_roles=absent_roles[absent_roles==1].reset_index()
absent_roles['AMOUNT']=0

absent_roles0=absent_roles.copy()
absent_roles0['ROLE']=0
absent_roles=absent_roles.append(absent_roles0)

amount=amount.append(absent_roles)
amount=amount.drop_duplicates(['SUBBRANCH_ID','DATE','HOUR','ROLE'])
amount=amount.sort_values(by=['SUBBRANCH_ID','DATE','HOUR'])

Получаем итоговый датасет

In [14]:
data_set=amount.copy()
data_set.set_index('DATE',inplace=True)
data_set.index = data_set.index.to_datetime()
data_set['MONTH']=data_set.index.month
data_set['DAY']=data_set.index.day
data_set['WEEKDAY']=data_set.index.weekday
data_set['IS_WEEKEND'] = data_set.WEEKDAY.isin([5,6])*1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: to_datetime is deprecated. Use pd.to_datetime(...)
  This is separate from the ipykernel package so we can avoid doing imports until


Меняем стобцы местами, делаем дату индексом и сортируем по дате датасет

In [15]:
data_set=data_set[['MONTH','DAY','WEEKDAY','IS_WEEKEND','HOUR','SUBBRANCH_ID','ROLE','AMOUNT']]
data_set['date']=data_set.index
data_set=data_set.sort_values(by=['date','HOUR']).drop(['date'],axis=1)
data_set.head(10)

,MONTH,DAY,WEEKDAY,IS_WEEKEND,HOUR,SUBBRANCH_ID,ROLE,AMOUNT
2013-07-01,7,1,0,0,8,1005625,0,21
2013-07-01,7,1,0,0,8,1005625,1,1
2013-07-01,7,1,0,0,8,1005839,0,5
2013-07-01,7,1,0,0,8,1005839,1,1
2013-07-01,7,1,0,0,8,1006034,0,19
2013-07-01,7,1,0,0,8,1006034,1,3
2013-07-01,7,1,0,0,8,1006255,0,12
2013-07-01,7,1,0,0,8,1006255,1,1
2013-07-01,7,1,0,0,9,1005625,0,34
2013-07-01,7,1,0,0,9,1005625,1,4


Разбиваем на обучающую и валидационную выборки и выделяем целевую переменную y 

In [500]:
test_size=0.3
test_index=int(len(data_set)*(1-test_size))
train=data_set[:test_index]
y_train=train['AMOUNT']
X_train=train.drop(['AMOUNT'],axis=1)
test=data_set[test_index:]
y_test=test['AMOUNT']
X_test=test.drop(['AMOUNT'],axis=1)

Добавляем фичи

In [ ]:
#Пробовал вводить разбиения по дням
'''def some_days(x):
    if x <11:
        return 0
    else:
        return 1
        
X_train['SOMEDAYS']=X_train['DAY'].apply(some_days)'''

# И по сезонам  
'''def season(x):
      return (x-1)//3
      
X_train['SEASON']=X_train['MONTH'].apply(season)'''

# Но результата не принесло

In [501]:
# Пробовал делать пересечения признаков (результат бустинга ухудшился, лин. регрессии немного улучшился)
'''categ_features = X_train.columns[X_train.dtypes == 'category']
for i, col1 in enumerate(categ_features):
    for j, col2 in enumerate(categ_features[i + 1:]):
        X_train[col1 + '_' + col2] = X_train[col1].astype(str) + '_' + X_train[col2].astype(str) 
        X_test[col1 + '_' + col2] = X_test[col1].astype(str) + '_' + X_test[col2].astype(str) '''

X_train_lin=X_train.copy()
X_test_lin=X_test.copy()

# Для лин. регрессии эти пересечения признаков улучшили результат        
second_col=['SUBBRANCH_ID','SUBBRANCH_ID','ROLE','HOUR','SUBBRANCH_ID']
for i, col1 in enumerate(['HOUR','ROLE','HOUR','WEEKDAY','WEEKDAY']):
    col2 = second_col[i]
    X_train_lin[col1 + '_' + col2] = X_train_lin[col1].astype(str) + '_' + X_train_lin[col2].astype(str) 
    X_test_lin[col1 + '_' + col2] = X_test_lin[col1].astype(str) + '_' + X_test_lin[col2].astype(str)
    
X_train_lin.head(5)

,MONTH,DAY,WEEKDAY,IS_WEEKEND,HOUR,SUBBRANCH_ID,ROLE,HOUR_SUBBRANCH_ID,ROLE_SUBBRANCH_ID,HOUR_ROLE,WEEKDAY_HOUR,WEEKDAY_SUBBRANCH_ID
2013-07-01,7,1,0,0,8,1005625,0,8_1005625,0_1005625,8_0,0_8,0_1005625
2013-07-01,7,1,0,0,8,1005625,1,8_1005625,1_1005625,8_1,0_8,0_1005625
2013-07-01,7,1,0,0,8,1005839,0,8_1005839,0_1005839,8_0,0_8,0_1005839
2013-07-01,7,1,0,0,8,1005839,1,8_1005839,1_1005839,8_1,0_8,0_1005839
2013-07-01,7,1,0,0,8,1006034,0,8_1006034,0_1006034,8_0,0_8,0_1006034


Делаем столбцы категориальными (для бустингов)

In [502]:
for col in X_train.columns:
    X_train[col]=X_train[col].astype('category')
for col in X_test.columns:
    X_test[col]=X_test[col].astype('category')

## Обучение

### Линейные регрессии

In [503]:
lr=LinearRegression()
tscv = TimeSeriesSplit(n_splits=5)
lscv=LassoCV(cv=tscv, n_jobs=-1, random_state=17)
rcv=Ridge(alpha=10)
ohe=OneHotEncoder(dtype=np.int,categorical_features='all')
pf=PolynomialFeatures()

In [345]:
# Пhобовал добавлять полиноминальные признаки, но они не дали почти прироста
'''pf_fit=pf.fit(X_train)
X_train=pf_fit.transform(X_train)
X_test=pf_fit.transform(X_test)'''

Делаю one hot encoding для категориальных признаков

In [504]:
fit_ohe=ohe.fit(X_train_lin)
X_o_train=fit_ohe.transform(X_train_lin)
X_o_test=fit_ohe.transform(X_test_lin)

In [505]:
X_o_train.shape

(55605, 287)

Ridge регрессия 

In [510]:
rcv.fit(X_o_train,y_train)
y_pred_rcv=rcv.predict(X_o_test)

In [511]:
y_pred_rcv=np.around(y_pred_rcv)
print('r2 = %f' % (r2_score(y_test, y_pred_rcv)))
print('MAE = %f' % (mean_absolute_error(y_test, y_pred_rcv)))

r2 = 0.787582
MAE = 5.594226


SVR

In [508]:
svr=SVR()
svr.fit(X_o_train,y_train)
y_pred_svr=svr.predict(X_o_test)

In [509]:
y_pred_svr=np.around(y_pred_svr)
print('r2 = %f' % (r2_score(y_test, y_pred_svr)))
print('MAE = %f' % (mean_absolute_error(y_test, y_pred_svr)))

r2 = 0.794316
MAE = 5.334145


### Catboost

Подбирал параметры с помощью GridSearchCV, самую успешную комбинацию для cat_features подбирал руками <br>
Улучшения и дополнительные признаки не помогли результату catboost, самый лучший результат получился на начальном датасете

In [ ]:
#tree_params={'l2_leaf_reg':[3,4,5,6,7]}
#dt_grid_search = GridSearchCV(cat, tree_params, n_jobs=-1, scoring ='neg_mean_squared_error', cv=tscv)
#dt_grid_search.fit(X_train,y_train)

In [530]:
cat=CatBoostRegressor(iterations=500, 
    random_seed=17 ,loss_function='RMSE',depth=8,learning_rate=0.01,random_strength=1,l2_leaf_reg = 6 , one_hot_max_size=1 ,
                       verbose=False)

In [531]:
cat.fit(X_train, y_train,use_best_model=True,  
cat_features=list([4,5]),
        eval_set=(X_test, y_test) )

In [532]:
y_pred=cat.predict(X_test)

Результат на отложенной выборке

In [533]:
y_pred=np.around(y_pred)
print('r2 = %f' % (r2_score(y_test, y_pred)))
print('MAE = %f' % (mean_absolute_error(y_test, y_pred)))

r2 = 0.838756
MAE = 4.590240


In [516]:
cat.feature_importances_

[1.1871125164392382,
 2.931933737860773,
 1.7585678997286018,
 0.6752033639746216,
 11.698305459349323,
 7.025792754213112,
 74.72308426843432]

Результат на кросс-валидации

In [534]:
-1*np.mean(cross_val_score(cat, X_train, y_train, cv=tscv, scoring='neg_mean_absolute_error'))

5.3112450285687745

# LightGBM

Использовал LGBM для дополнительной проверки результатов, и для быстрой проверки новых признаков <br>
LGBM отработал хуже чем catboost, зато намного быстрее выполняется обучение модели

In [ ]:
# LGBM работает только с числовыми данными, поэтому приходилось перекодировать признаки в процессе тестирования
'''le=LabelEncoder()
for i in X_train.columns[9:]:
    X_train[i]=le.fit_transform(X_train[i])
for i in X_test.columns[9:]:
    X_test[i]=le.fit_transform(X_test[i])'''

In [522]:
L_train=lgb.Dataset(X_train,label=y_train)
L_test=lgb.Dataset(X_test,label=y_test)
lgbparam                 = {}
lgbparam['metric']       = 'rmse'
lgbparam['application']  = 'regression'
lgbparam['nthread']      = 6
lgbparam['num_boost_round']=1000
lgbparam['max_depth']=5
#lgbparam['max_cat_to_onehot'] = 2
#lgbparam['learning_rate'] = 0.05
#lgbparam['boosting']         = 'rf'


In [523]:
lgb_model = lgb.train(lgbparam, L_train,valid_sets=[L_test], verbose_eval=10,early_stopping_rounds=1)
lgb_pred=lgb_model.predict(X_test)

Training until validation scores don't improve for 1 rounds.
[10]	valid_0's rmse: 8.83993
[20]	valid_0's rmse: 7.35374
[30]	valid_0's rmse: 7.21964
Early stopping, best iteration is:
[29]	valid_0's rmse: 7.2022


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Результат на отложенной выборке

In [524]:
lgb_pred=np.around(lgb_pred)
print('r2 = %f' % (r2_score(y_test, lgb_pred)))
print('MAE = %f' % (mean_absolute_error(y_test, lgb_pred)))

r2 = 0.815824
MAE = 5.192606


Почему-то lgbm совсем по другому расставил важность признакам

In [525]:
sorted(list(zip(X_train.columns,lgb_model.feature_importance())),key=lambda x: x[1])

[('ROLE', 29),
 ('WEEKDAY', 62),
 ('IS_WEEKEND', 84),
 ('MONTH', 135),
 ('SUBBRANCH_ID', 161),
 ('HOUR', 175),
 ('DAY', 179)]

Результат на кросс-валидации

In [526]:
%%time
cv_scores = []
for train_idx, test_idx in tscv.split(X_train, y_train):
    cv_train_df, cv_valid_df = X_train.iloc[train_idx, :], X_train.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    L_train=lgb.Dataset(cv_train_df,label=y_cv_train)
    lgb_model = lgb.train(lgbparam, L_train)
    
    cv_scores.append(mean_absolute_error(y_cv_valid, lgb_model.predict(cv_valid_df,num_iteration =-1)))

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:681: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Wall time: 678 ms


In [527]:
np.mean(cv_scores)


5.263305042686472

В итоге лучшей моделью оказался catboost в данной задаче <br>
